# load packages

In [29]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import preprocessing

# Load dataset and Data Processing

In [15]:
df = pd.read_csv('Audobook_data.csv', header=0)

In [16]:
df.columns = ['ID', 'Book length overall', ' Book length avg', 'price overall', 'price avg', 'reviewed','ReviewRate','minutes listened','completion','support request','last visit minus purchase date', 'Targets'
]

In [17]:
print(df.shape)
df.head()

(14083, 12)


,ID,Book length overall,Book length avg,price overall,price avg,reviewed,ReviewRate,minutes listened,completion,support request,last visit minus purchase date,Targets
0,611,1404.0,2808,6.66,13.33,1,6.5,0.0,0.0,0,182,1
1,705,324.0,324,10.13,10.13,1,9.0,0.0,0.0,1,334,1
2,391,1620.0,1620,15.31,15.31,0,9.0,0.0,0.0,0,183,1
3,819,432.0,1296,7.11,21.33,1,9.0,0.0,0.0,0,0,1
4,138,2160.0,2160,10.13,10.13,1,9.0,0.0,0.0,0,5,1


In [35]:
df.shape

(14083, 12)

In [23]:
df.describe()

,ID,Book length overall,Book length avg,price overall,price avg,reviewed,ReviewRate,minutes listened,completion,support request,last visit minus purchase date,Targets
count,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000,14083.000000
mean,16773.620535,1591.241302,1678.574451,7.103576,7.543621,0.160761,8.909795,0.125668,118.595166,0.070227,61.939431,0.158773
std,9691.225165,504.335798,654.849284,4.931782,5.560284,0.367324,0.643429,0.241212,268.739618,0.472173,88.209221,0.365477
min,2.000000,216.000000,216.000000,3.860000,3.860000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8371.500000,1188.000000,1188.000000,5.330000,5.330000,0.000000,8.910000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,16715.000000,1620.000000,1620.000000,5.950000,6.070000,0.000000,8.910000,0.000000,0.000000,0.000000,11.000000,0.000000
75%,25187.500000,2160.000000,2160.000000,8.000000,8.000000,0.000000,8.910000,0.130000,64.800000,0.000000,105.000000,0.000000
max,33683.000000,2160.000000,7020.000000,130.940000,130.940000,1.000000,10.000000,1.000000,2116.800000,30.000000,464.000000,1.000000


In [24]:
df['Targets'].value_counts()

0    11847
1     2236
Name: Targets, dtype: int64

In [66]:
raw_csv_data = np.loadtxt('Audobook_data.csv', delimiter = ',')

In [67]:
print(raw_csv_data.shape)

(14084, 12)


In [68]:
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

In [69]:
# Balance the dataset
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []
for i in range(targets_all.shape[0]):
    if targets_all[i] ==0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete (targets_all, indices_to_remove, axis=0)

In [70]:
# Scale inputs
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

# Shuffle data
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [71]:
# Split data into train, validation, test
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1783.0 3579 0.4981838502374965
220.0 447 0.49217002237136465
234.0 448 0.5223214285714286


In [72]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)